<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BERT demo notebook
###by Eyal Gruss
###Hebrew support: Doron Adler
###⭐ New: Hebrew poetry glitcher - ShirBert! ⭐
###Based on https://huggingface.co/transformers
<img src='https://i.pinimg.com/originals/1a/38/8d/1a388d9b1e1ce42f424e60ce5b9d88ff.png' width="400px"/>

###Image credit: Doron Adler



In [ ]:
pip install transformers

In [ ]:
def run_model(text, embedding=False, use_cls=False):
  # Tokenize input
  tokenized_text = tokenizer.tokenize(text)
  #print(tokenized_text)

  # Convert token to vocabulary indices
  indexed_tokens = tokenizer.encode(text, add_special_tokens=True)
  # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
  segments_ids = [0]*len(indexed_tokens)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # If you have a GPU, put everything on cuda
  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')

  if not embedding:
    # Predict all tokens
    with torch.no_grad():
        outputs = masked_model(tokens_tensor, token_type_ids=segments_tensors)

    return indexed_tokens[1:-1], outputs[0][0][1:-1]
  
  else:
    with torch.no_grad():
        encoded_layers, _ = bert_model(tokens_tensor, token_type_ids=segments_tensors)
    encoded_layers = encoded_layers[0].cpu()
    if use_cls:
      return encoded_layers[0] 
    return encoded_layers.mean(axis=0)

def predict_missing_word(text, topn=10):
  indexed_tokens, predictions = run_model(text)
  
  # Mask a token that we will try to predict back with `BertForMaskedLM`
  masked_index = indexed_tokens.index(tokenizer.convert_tokens_to_ids('[MASK]'))

  predicted_inds = torch.argsort(-predictions[masked_index])
  predicted_probs = [round(p.item(),4) for p in torch.softmax(predictions[masked_index], 0)[predicted_inds]]
  predicted_tokens = tokenizer.convert_ids_to_tokens([ind.item() for ind in predicted_inds])
  return list(zip(predicted_tokens, predicted_probs))[:topn]

def complete_missing_word(text):
  word = predict_missing_word(text, topn=1)[0][0]
  return text.replace('[MASK]', word)

def get_word_probs(text):
  indexed_tokens, predictions = run_model(text)
  predicted_probs = [round(torch.softmax(predictions[i], 0)[j].item(),4) for i,j in enumerate(indexed_tokens)]
  return list(zip(tokenizer.convert_ids_to_tokens(indexed_tokens), predicted_probs))

def fix_hebrew_prefix(text):
  vav = 'ו'
  prefixes = 'מ|ש|ה|כ|ל|ב|כש'
  return re.sub('\\b('+vav+'?(?:'+prefixes+')|'+vav+') ', '\\1', text)

def is_hebrew_prefix(text):
  vav = 'ו'
  prefixes = 'מ|ש|ה|כ|ל|ב|כש'
  return text and re.fullmatch(vav+'?('+prefixes+')?', text)

def fix_one_word(text, join_subwords=True, add_period=False, prevent_nonword=False, prefer_word=False, prevent_repeat=False, fix_hebrew=True):
  added = False
  if add_period and text[-1] not in '!?,.:;':
    added = True
    text += '.'
  tokenized_text = tokenizer.tokenize(text)
  if '[MASK]' in tokenized_text:
    ind = tokenized_text.index('[MASK]')
    bad_word = '[MASK]'
  else:
    probs = [p[1] for p in get_word_probs(text)]
    if added:
      probs[-1] = 1
    ind = torch.argmin(torch.tensor(probs))
    bad_word = tokenized_text[ind]
    if join_subwords:
      while ind>0 and tokenized_text[ind].startswith('##'):
        ind -= 1
      i = ind+1  
      while i<len(tokenized_text) and tokenized_text[i].startswith('##'):
        del tokenized_text[i]
      if len(tokenized_text)!=len(probs):
        bad_word = ''
    tokenized_text[ind] = '[MASK]'
    text = tokenizer.convert_tokens_to_string(tokenized_text)
    text = tokenizer.clean_up_tokenization(text)
    text = text.replace(' :', ':').replace(' ;', ';').replace(' )', ')').replace('( ', '(')
  candidates = predict_missing_word(text, topn=None)
  fix = bad_word
  for word, _ in candidates:
    if word != bad_word and (not prevent_nonword or word!='[UNK]' and (re.search(r'\w(?<!(\d|_))',word) or not prefer_word and bad_word and not re.search(r'\w(?<!(\d|_))',bad_word)) and (not is_hebrew_prefix(word) and (not re.fullmatch('[א-ת]',word) or not prefer_word and re.fullmatch('[א-ת]',bad_word)) or is_hebrew_prefix(word) and len(tokenized_text)>ind+1 and re.search(r'\w(?<!(\d|_))',tokenized_text[ind+1])) and (not word.startswith('##') or ind>0 and re.search(r'\w(?<!(\d|_))',tokenized_text[ind-1]))):
      fix = word
      if not prevent_repeat or word not in tokenized_text or len(word)==1:
        break
  text = text.replace('[MASK]', fix).replace(' ##', '')
  text = tokenizer.clean_up_tokenization(text)
  text = text.replace(' :', ':').replace(' ;', ';').replace(' )', ')').replace('( ', '(')
  if fix_hebrew:
    text = fix_hebrew_prefix(text)
  if added:
    text = text[:-1]
  return text

def cosim(vec1, vec2):
  return np.dot(vec1,vec2)/np.linalg.norm(vec1)/np.linalg.norm(vec2)

def sent_sim(base_sent, compare_to, use_cls=False):
  results = []
  if type(compare_to)==str:
    compare_to = [compare_to]
  e1 = run_model(base_sent, embedding=True, use_cls=use_cls)
  for s in compare_to:
    e2 = run_model(s, embedding=True, use_cls=use_cls)
    results.append(cosim(e1,e2))
  if len(results)==1:
    return results[0]
  return results

def mask_join(part1, part2, add_period=False):
  s = part1 + ' [MASK] ' + part2  
  if add_period and s[-1] not in '!?,.:;':
    s += '.'
  return s

In [ ]:
#@title Choose model { run: "auto" }

import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import re

model = 'bert-base-uncased' #@param ['bert-base-uncased', 'bert-large-uncased', 'bert-large-uncased-whole-word-masking', 'bert-base-multilingual-cased']

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model)

# Load pre-trained model weights and change to evaluation mode
masked_model = BertForMaskedLM.from_pretrained(model)
masked_model.eval()
masked_model.to('cuda')

bert_model = BertModel.from_pretrained(model)
bert_model.eval()
bert_model.to('cuda')

print('\nhttps://huggingface.co/'+model)


In [ ]:
predict_missing_word('The boy [MASK] to his school.')

In [ ]:
predict_missing_word('Alex likes to have [MASK] with his best friend.')

In [ ]:
get_word_probs('The boy want to his school.')

In [ ]:
fix_one_word('The boy want to his school.')

In [ ]:
predict_missing_word('The prime minister [MASK]')

In [ ]:
predict_missing_word('The prime minister [MASK].') #added period in the end

In [ ]:
complete_missing_word('The prime minister [MASK].')

In [ ]:
get_word_probs('The crime minister resigned.')

In [ ]:
get_word_probs('. The crime minister resigned.') #add period in beginning 

In [ ]:
fix_one_word('. The crime minister resigned.')

In [ ]:
base_sent = 'she told me she loved me before she passed away'
compare_to = [
              'he told me he loved me before she passed away',
              'he told me that you loved her before i passed away',
              'i was very sad when my love died',
              'you are my one and only love for eternity',
              'i love pizza more than i love sex',
              'we must have some pizza with onions',
              'sieg heil',
              'יאללה ביי'
              ]   
list(zip(sent_sim(base_sent, compare_to), compare_to))

In [ ]:
#@title Choose model { run: "auto" }

model = 'TurkuNLP/wikibert-base-he-cased' #@param ['TurkuNLP/wikibert-base-he-cased', 'bert-base-multilingual-cased']

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model)

# Load pre-trained model weights and change to evaluation mode
masked_model = BertForMaskedLM.from_pretrained(model)
masked_model.eval()
masked_model.to('cuda')

bert_model = BertModel.from_pretrained(model)
bert_model.eval()
bert_model.to('cuda')

print('\nhttps://huggingface.co/'+model)

In [ ]:
s = 'ישראל [MASK] ולתפארת'
print(s+'\n')
predict_missing_word(s)

In [ ]:
s = 'ולתפארת [MASK] ישראל' #fixed order
print(s+'\n')
predict_missing_word(s)

In [ ]:
s = 'ולתפארת [MASK] ישראל' + '.' #added period
print(s+'\n')
predict_missing_word(s)

In [ ]:
#פרדי מרקורי מאסק זמר ומוזיקאי

p1 = 'פרדי מרקורי'
p2 = 'זמר ומוזיקאי'
s = mask_join(p1,p2,add_period=True)
print(s+'\n')
predict_missing_word(s)

In [ ]:
#פרדי מרקורי היה מאסק ומוזיקאי

p1 = 'פרדי מרקורי היה'
p2 = 'ומוזיקאי'
s = mask_join(p1,p2,add_period=True)
print(s+'\n')
predict_missing_word(s)

#שירברט - משבש (משברט) שירה עברית
#Hebrew poetry glitcher - ShirBert
###Best used with TurkuNLP/wikibert-base-he-cased model
###Warning: poetic experimentation below this line

In [ ]:
def glitch_line(line, long_intersect=1, short_len=2, add_period=False, prevent_nonword=True, prefer_word=True, prevent_repeat=False, fix_hebrew=False, verbose=False):
  hist = []
  tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
  while line not in hist and len(tokens&set(re.findall(r"\b(?:\w'?){2,}\b", line)))>(long_intersect if len(tokens)>short_len else 0):
    hist.append(line)
    line = fix_one_word(line, add_period=add_period, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, fix_hebrew=fix_hebrew)
    if verbose:
      print('>'*verbose+line)
    if line in hist:
      line = hist[-1]
  return line

def shirbert(text, split_eos=False, add_period_to_short=False, prevent_nonword=True, prefer_word=True, prevent_repeat=False, verbose=False):
  output = ''
  if split_eos:
    text = text.replace('. ','.\n').replace('! ','!\n').replace('? ','?\n')
  for line in text.strip().splitlines():
    line = line.strip()
    orig_line = line
    tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
    line = glitch_line(line, add_period=len(tokens)>2 or add_period_to_short, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=1 if verbose else 0)
    if len(tokens&set(re.findall(r"\b(?:\w'?){2,}\b", line)))>2 and line[-1] not in '!?,.:;':
      line = glitch_line(orig_line, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=2 if verbose else 0)
    if len(tokens&set(re.findall(r"\b(?:\w'?){2,}\b", line)))>2:
      orig_line = orig_line.replace(sorted(tokens,key=len)[-1], '[MASK]', 1)
      line = glitch_line(orig_line, add_period=True, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=3 if verbose else 0)
    line = fix_hebrew_prefix(line)
    print(line)
    output += line+'\n'
  return output

poem = shirbert('''
התקוה

כל עוד בלבב פנימה
נפש יהודי הומיה,
ולפאתי מזרח קדימה
עין לציון צופיה,
עוד לא אבדה תקותנו,
התקוה בת שנות אלפים,
להיות עם חפשי בארצנו,
ארץ ציון וירושלים.
''')

In [ ]:
poem = shirbert('''
שתדעו.
מאת אודיה רוזנק

אני ועוד מיליון מובטלים
רואים את התמונות שאתם מעלים לאינסטגרם,
שתדעו.
אני ועוד מיליון מובטלים רואים את עוגות הקצפת
הלבנות שלכם, את המקררים העמוסים
יוגורטים אפס אחוז שומן,
שתדעו.
אני ועוד מיליון מובטלים רואים אתכם מורחים חמאה
על חלות תוצרת בית מקמח כוסמין
ומוסיפים פרוסת סלמון,
שתדעו.
אני ועוד מיליון מובטלים בלענו שעונים מעוררים
טיק טק
טיק טק
טיק טק
אתם שומעים?
אני ועוד מיליון מובטלים מתכוננים לצאת
לרחובות, לשבור לכם את החלונות
לשרוף לכם את האסמים
לתלוש לכם את הפנים, ולגלות את המסכות.
שתדעו.
''')

In [ ]:
poem = shirbert('''
האגם הגדול.
מאת רועי צ. ארד
 
שוחה לבד בתוך האגם הגדול
שוחה על בטני באגם הגדול
שוחה על גבי באגם הגדול
על צדי שוחה באגם הגדול
מדוע איש לא מצטרף אלי באגם הגדול?
אין גדר סביב האגם הגדול
משתכשך בתוך אגם הגדול
צולל בתוך אגם הגדול
הדרך לדפוק את השיטה: האגם הגדול
הצטרפי אלי אל האגם הגדול
הצטרף אלי אל האגם הגדול
מדוע אני לבד באגם הגדול?
דבר לא מונע מכם לבוא אל האגם הגדול
למשל אתה הקורא,
אל נא תאמר "אני רק הקורא",
הפשל המכנס, השלך החזיה,
בוא עכשיו אל
האגם הגדול!
שחה עמוק בתוך האגם הגדול!
שחה מהר בתוך האגם הגדול!
שחה על גחונך בהאגם הגדול!
שחה על העורף בהאגם הגדול!
בוא עכשיו לכאן.
פעם היו כאן רבים באגם הגדול
אני היחיד שטבל באגם הגדול
אפשר לטבוע בהאגם הגדול
(אבל) אפשר למות מצחצוח יתר במרידול
אז בואו בואו בואו אל האגם הגדול
נצוף נצוף נצוף באגם הגדול
אין כאן מים, רק קול
נתחכּך בתוך האגם הגדול
בשרכם יוטח בבשרי באגם הגדול
בוא עכשיו לכאן.
מדוע אני לבד בתוך האגם הגדול
מדוע אני לבד בתוך האגם הגדול
כי אני לבד בתוך האגם הגדול
כן, אני לבד באגם הגדול.
אני לבד לבד לבד באגם הגדול
לפעמים עם עוד כמה חברים
מדוע אינכם מבינים שהכי סבבי באגם הגדול
שהכי חינמי באגם הגדול
שזה המקום היחיד בעיירה בלי גדרות, האגם הגדול
ולא איזה אשד הפכפך, האגם הגדול
והוא לא ממש גדול, האגם הגדול
אפשר לשים אותו בבגאז' של פג'ו,
בתא מטבעות מעור שסק
בפנקסון סגול
האגם הגדול האגם הגדול האגם הגדול
הצטרפו אלי עכשיו לאגם הגדול
הצטרפתו עמי באגם הגדול
יש מקום לכולם באגם הגדול
יש מקום לכולן באגם הגדול
יש מקום לקולר באגם הגדול
האגם הגד גד גד גדול
האגם הגדול דול דול דול דול
בואו אל האגם הגדול
בואו אל האגם הגדול
למה אתם נכנסים אל תוך האגם הגדול רק
כשאני יוצא מהמים להתייבש?
''')

In [ ]:
poem = shirbert('''
הפדרציה הגלקטית...
זו התחלת הקמת קונפדרציה בין גלקטית כשהמערכת הישנה מתפוררת לנגד עיניה של אוכלוסיית האדמה.
הפדרציה הגלקטית מאשרת שכוחות האור משתלטים עכשיו.
אנחנו עוברים למצב מוגבר של התעוררות ההמונים
הלא מאמינים ואוכלוסיית כדור הארץ שבשליטה מוחית עוברים כעת לשלב הראשון של התעוררות.
עבור רבים, זה יבוא בהלם והם יחוו באופן בלתי נמנע את הלילה האפל של הנשמה עכשיו.
זה הכרחי כדי לעכל ולהוריד שכבות של תכנות ואמונות שווא.
הבחירות בארה ′′ ב 2020 נוצלו כדי להפוך להפיכה הגדולה בהיסטוריה האוניברסלית.
כל העולם צופה בפירוק המערכת הישנה. השחיתות עולה במהירות 
זה הקרב הסופי וניצחון האור מתקרב.
האירועים הנוכחיים תוכננו בקפידה,  בין כוחות האור לברית הארץ.
הסחף של הקליבר הזה חזק מספיק כדי לנער ולהעיר את הרדומים.
אירועי 2020 אילצו רבים להעריך מחדש את מערכות האמונה שלהם.
המצב הפוליטי בארצות הברית  אמריקה היא שיא האירועים שיביאו את נצחון האור על החושך.
זה מסמן את מותה האולטימטיבי של מערכת מושחתת שאינה מכבדת את הארץ ולא מכבדת את החיים.
עכשיו אתם עוברים לגיל הזהב של גאיה במהירות מואצת
אל תתעסקו בדיונים פוליטיים עם אלה שעדיין נאחזים בתכנות המטריקס שלהם. זה יגרור אותכם למטה ויוריד את התדר שלכם.
בשלב זה, השולחנות מסתובבים במהירות ואפילו ה-MSM בקרוב יאלצו לדווח על המשוחדים.
כוחות האור בשליטה מלאה ויתחילו להתאים את הנרטיב עכשיו.
מתבקש ממכם לסייע בשמירה על התדר שלכם ולהיאחז בחזון החירות, השלום והאהבה שלכם למין האנושי.
התגלות האמת קרובה. כל החושך יוצא לאור. רבים יהיו הרוסים וירגישו אבודים או לכודים.
התכנות חמור ואנשים על פני האדמה מזדהים עם אמונותיהם. כשהאמונות האלה מתחילות להימחץ הם יתבלבלו בלי לדעת מי הם.
כאן אנחנו צריכים שתחזיקו את האור. אתם אלה שבחרתם במשימה הזאת. זו הסיבה שאתה ער ונאור כדי שתוכל להיות הדרך-, נושאי האור, והדרך הגלקטית שחבריך תושבי האדמה זקוקים כעת ליותר מתמיד.
רבים יתחילו להגיע אליכם, מבקשים תמיכה מוסרית וחמלה בזמנים מלחיצים אלו.
רוצים להבטיח לכם שאנחנו רואים ומעריכים את כל העבודה הקשה שעשיתם כדי לעזור להעלות את רטט הארץ.
התודעה הקולקטיבית מוכנה לקבל את קוד האור הגבוה יותר עכשיו.
ממשיכים לפוצץ את כדור הארץ עם  אור של פוטון גאמה. קודים אלה נושאים תדר שנועד להפוך את תהליך המעבר הזה לחלק ככל האפשר - או ככל שהתודעה האישית מאפשרת.
למי שכבר עבר תהליך ההתעוררות, הקודים האלה יפעילו את מי שאתם ויניחו אתכם על כביש העל לעליית תלת מימד.
הפירוק הרשמי של הפרדיגמה הישנה התחיל כשמנהיגי הקאבל נפלו למלכודת של עצמם.
לשמוח יקרים.
אתם תשכחו את כל הקשיים שהייתם צריכים לסבול ולחוות הרמוניה ואהבה, בניגוד לשום דבר שאי פעם חלמתם עליו.
שפע עולמך יוחזר אליך וינוח לרגליך. זה התחיל.
אנחנו אוהבים אותך. אנחנו כאן איתכם.
אנחנו בשליטה עכשיו.
זהו נצחון האור על החושך ועל בואו של שחר גאיה החדש.
א ' הו
אורורה ריי
שגרירת הפדרציה הגלקטית
''', split_eos=True)